In [3]:
####################################
# IMPORTS:
####################################

# External dependencies:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from scipy.spatial.distance import cosine, euclidean, cdist
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModel, get_scheduler
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# Internal libraries:
import numpy as np
import time
import random
import os
from os import listdir, write
from os.path import isfile, join, splitext
from data_loader import file_list_loader, norm_list_loader, mention2concept, encoder, Dataset
from preprocess import id_combination, lowercaser_mentions
from normalization import NeuralNetwork, cos_dist
from inference import tokenize, inference
import my_global
 
my_global._init()

In [4]:
train_file = file_list_loader('./dataset/train/train_file_list.txt')
print(train_file)


{'0034': None, '0038': None, '0062': None, '0070': None, '0086': None, '0090': None, '0094': None, '0098': None, '0130': None, '0174': None, '0214': None, '0222': None, '0250': None, '0266': None, '0278': None, '0286': None, '0302': None, '0314': None, '0318': None, '0334': None, '0338': None, '0358': None, '0362': None, '0370': None, '0390': None, '0410': None, '0428': None, '0431': None, '0461': None, '0463': None, '0467': None, '0468': None, '0476': None, '0477': None, '101407944_PUMC': None, '105732749': None, '156406283': None, '176318078_a': None, '262912613': None, '284487129': None, '320422564': None, '332803550': None, '433651389': None, '498710998': None, '517414339': None, '622086964': None, '638157550_SC': None, '723989226': None, '974381789': None, '989519730_WGH': None}
{'989519730_WGH': None, '0476': None, '0362': None, '0214': None, '262912613': None, '0431': None, '0314': None, '0428': None, '0062': None, '0370': None, '176318078_a': None, '0034': None}


In [12]:
####################################
# LOADING DATA
####################################

# cui lists in training set
norm_list = norm_list_loader('./dataset/train/train_norm.txt')
# file names
train_file = file_list_loader('./dataset/train/train_file_list.txt')
test_file = file_list_loader('./dataset/test/test_file_list.txt')

####################
# 1/4 of training set
####################
# Select 1/4 of the keys randomly
selected_keys = random.sample(list(train_file.keys()), len(train_file)//4)

# Create a new dictionary with only the selected keys
train_small_file = {k: train_file[k] for k in selected_keys}
norm_list = {k: train_file[k] for k in selected_keys}

####################
# 1/4 of test set
####################
# Select 1/4 of the keys randomly
selected_keys = random.sample(list(test_file.keys()), len(test_file)//4)

# Create a new dictionary with only the selected keys
test_small_file = {k: test_file[k] for k in selected_keys}

train_norm, train_cui_less_dict, train_span_split = mention2concept('./dataset/train/train_note', './dataset/train/train_norm', train_small_file, with_text = False)
test_norm, test_cui_less_dict, test_span_split = mention2concept('./dataset/test/test_note', './dataset/test/test_norm_cui_replaced_with_unk', test_small_file, with_text = False)
####################################
# PRE-PROCESSING
####################################

train_dict = id_combination(train_norm)
train_dict = lowercaser_mentions(train_dict)

test_dict = id_combination(test_norm)
test_dict = lowercaser_mentions(test_dict)

In [13]:
train_span_split 

{'2': 2, '1': 18, '0': 1608}

In [4]:
################################################
# INITIALIZING
################################################

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
my_global.set_value('device', device)
print(f"Using {device} device")

Using cuda device


In [5]:
################################################
# LOADING EMBEDDING MODEL
################################################

model_name = 'dmis-lab/biobert-base-cased-v1.1'
model = AutoModel.from_pretrained(model_name).to(device)
embbed_size = 768
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 20

my_global.set_value('model', model)
my_global.set_value('tokenizer', tokenizer)
my_global.set_value('max_length', max_length)
my_global.set_value('embbed_size', embbed_size)

X_train, y_train = encoder(train_dict, tokenizer)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of mentions: 6533


In [7]:
################################################
# PREPARING DATA
################################################

train_set = Dataset(X_train, y_train)

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)

In [8]:
################################################
# TRAINING
################################################

# fine-tuning layer (linear)
basenorm = NeuralNetwork(embbed_size).to(device)

# training parameters
learning_rate = 1e-5
epochs = 50
optimizer = torch.optim.NAdam(basenorm.parameters(), lr=learning_rate)
num_training_steps = epochs * len(train_dataloader)

# loss function
loss_fn = cos_dist
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [31]:
i = 0
for X, y in train_dataloader:
    for tokenized_mention, tokenized_label in zip(X, y):
        tokenized_mention = tokenized_mention.to(device)
        output = model(tokenized_mention, output_hidden_states=True)
        print(output.last_hidden_state[0])
        print(output[0][:,0])


tensor([[-0.0012,  0.2092, -0.4061,  ..., -0.2054, -0.0355,  0.0053],
        [-0.3573, -0.0947, -0.2851,  ...,  0.1545,  0.1096, -0.1433],
        [-0.1026,  0.7099, -0.0197,  ...,  0.0155,  0.0062, -0.0313],
        ...,
        [-0.4421, -0.2894,  0.2135,  ...,  0.0467,  0.4816, -0.3530],
        [-0.4606, -0.3889,  0.2401,  ...,  0.0515,  0.4944, -0.2978],
        [-0.4516, -0.3765,  0.1843,  ...,  0.0273,  0.6031, -0.3405]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[-1.2276e-03,  2.0920e-01, -4.0613e-01, -1.5941e-01, -3.0139e-01,
          6.9470e-02,  1.8273e-01, -9.4531e-02,  1.1949e-02,  3.7418e-01,
          3.1032e-01,  2.7529e-02, -6.5090e-01, -5.0514e-02, -2.5541e-01,
          4.0708e-01, -1.3795e-01, -5.5265e-01, -5.6462e-01,  2.0931e-01,
          1.6665e-02, -4.6080e-01, -2.1817e-01,  4.9839e-02, -1.1101e-01,
         -2.5037e-01,  1.6001e-01,  5.6481e-03, -3.1682e-01,  5.6493e-01,
         -5.5967e-02,  3.2998e-01, -3.5887e-01, -6.0016e-01, -1.2958e-0

KeyboardInterrupt: 

In [8]:
# training loop
model.train()
basenorm.train()

def checkpoint_loader(checkpoint):
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    batch_loss = checkpoint['loss']
    return epoch, model, optimizer, batch_loss

def train(model, optimizer,from_checkpoint = False):
    if from_checkpoint:
        epoch = input("Please enter a number of epoch to load model: ")
        checkpoint = torch.load(f'./checkpoint/epoch{epoch}_checkpoint.pt')
        start_epoch, model, optimizer, batch_loss = checkpoint_loader(checkpoint)
    else:
        start_epoch = 0

    start_time = time.time()
    for epoch in range(start_epoch, epochs):
        for X, y in train_dataloader: # both X and y contains n=batch_size tokenized mentions and labels respectively
            batch_loss = None
            for tokenized_mention, tokenized_label in zip(X, y):
                tokenized_mention = tokenized_mention.to(device)
                tokenized_label = tokenized_label.to(device)
                pred = basenorm(model(tokenized_mention)[0][:,0]) # Taking last hidden state of the embedding model and piping it into a linear layer.
                ground_truth = basenorm(model(tokenized_label)[0][:,0])
                loss = loss_fn(pred, ground_truth) # Cosine similarity between embedding of mention and associated label.
                if batch_loss == None:
                    batch_loss = loss.reshape(1,1)
                else:
                    batch_loss = torch.cat((batch_loss, loss.reshape(1,1)), dim=1) # Appends current loss to all losses in batch

            # Backpropagation
            batch_loss = torch.mean(batch_loss) # Averages loss over the whole batch.
            batch_loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        end_time = time.time()
        print("Epoch [{}/{}], Elapsed Time: {:.2f}s".format(epoch+1, epochs, end_time - start_time))
        print(f"loss = {batch_loss.item()}")

        if (epoch+1) % 10 == 0:
            # Check if the directory already exists
            if not os.path.exists('./checkpoint'):
            # If the directory does not exist, create it
                os.makedirs('./checkpoint')
            checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': batch_loss
                        }
            # Save the checkpoint
            torch.save(checkpoint, f'./checkpoint/epoch{epoch}_checkpoint.pt')

In [22]:
# starting training
# train(model, optimizer, from_checkpoint=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Elapsed Time: 398.32s
loss = -0.8204929232597351


  4%|▍         | 2/50 [12:33<4:58:20, 372.93s/it]

Epoch [2/50], Elapsed Time: 354.86s
loss = -0.9015124440193176


  6%|▌         | 3/50 [18:03<4:36:37, 353.13s/it]

...

Epoch [48/50], Elapsed Time: 318.78s
loss = -0.985721230506897


 98%|█████████▊| 49/50 [5:02:24<05:19, 319.18s/it]

Epoch [49/50], Elapsed Time: 318.03s
loss = -0.9945096969604492


100%|██████████| 50/50 [5:07:44<00:00, 369.28s/it]

Epoch [50/50], Elapsed Time: 319.43s
loss = -0.9929291605949402


In [9]:
################################################
# TEST
################################################

# load the model
#model.eval()
basenorm.eval()
epoch = input("Please enter a number of epoch to load model: ")
checkpoint = torch.load(f'./checkpoint/epoch{epoch}_checkpoint.pt')
start_epoch, model, optimizer, batch_loss = checkpoint_loader(checkpoint)
model.eval()

In [10]:
norm_list.remove('CUI-less')
dd_predictions = inference(norm_list=norm_list, basenorm=basenorm, dd_test=test_dict)

Embedding ontology concept labels...
Number of concepts in ontology: 6683
Done.



Building embeddings from cui list: 0it [00:00, ?it/s]/mnt/data/yangye/MCN-bertnorm/inference.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_mention = torch.tensor(tokenize(dd_test[id]['mention']).to(device))
Building embeddings from cui list: 6925it [00:53, 129.86it/s]


	Distance matrix calculation...
	Done.


In [11]:
def tokenize(sentence):
    max_length = my_global.get_value('max_length')
    tokenizer = my_global.get_value('tokenizer')
    device = my_global.get_value('device')
    return tokenizer.encode(sentence, padding="max_length", max_length=max_length, truncation=True, add_special_tokens=True, return_tensors="pt").to(device) # Tokenize input into ids.

print("Embedding ontology concept labels...")

Embedding ontology concept labels...


In [12]:
######
# Build labels/tags embeddings from ontology:
######
del norm_list
norm_list = norm_list_loader('./dataset/train/train_norm.txt')
norm_list.remove('CUI-less')
norm_list = set(norm_list)
cui_encode = dict()
N = 0
with torch.no_grad():
    for cui in norm_list:
        cui_encode[cui] = basenorm(model(tokenize(cui))[0][:,0]).cpu().detach().numpy()
        N += 1
        if cui == 'CUI-less':
            print('yes')
        if embbed_size == None:
            embbed_size = len(cui_encode[cui][0])
print("Number of concepts in ontology:", len(norm_list))
print("Done.\n")

yes
Number of concepts in ontology: 2331
Done.



In [19]:
len(cui_encode.keys())

2331

In [13]:
######
# Build mention embeddings from testing set:
######

X_pred = np.zeros((len(test_dict.keys()), embbed_size)) # (6925, 768)
with torch.no_grad():
    for i, id in tqdm(enumerate(test_dict.keys()), desc ='Building embeddings from cui list'):
        tokenized_mention = torch.tensor(tokenize(test_dict[id]['mention']).to(device))
        X_pred[i] = basenorm(model(tokenized_mention)[0][:,0]).cpu().detach().numpy()

Building embeddings from cui list: 0it [00:00, ?it/s]/tmp/ipykernel_1363123/56327478.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_mention = torch.tensor(tokenize(test_dict[id]['mention']).to(device))
Building embeddings from cui list: 6925it [00:53, 130.26it/s]


In [20]:
test_dict

{'0002_N000': {'cui': 'unk', 'mention': 'hiv positive'},
 '0002_N001': {'cui': 'unk', 'mention': 'left upper quadrant pain'},
 '0002_N002': {'cui': 'unk', 'mention': 'nausea'},
 '0002_N003': {'cui': 'unk', 'mention': 'vomiting'},
 '0002_N004': {'cui': 'unk', 'mention': 'a long-standing complaint'},
 '0002_N005': {'cui': 'unk', 'mention': 'hiv positive'},
 '0002_N006': {'cui': 'unk', 'mention': 'blood transfusions'},
 '0002_N008': {'cui': 'unk', 'mention': 'cat scratch fever'},
 '0002_N009': {'cui': 'unk', 'mention': 'resection'},
 '0002_N010': {'cui': 'unk',
  'mention': 'an abscess in the left lower extremity'},
 '0002_N011': {'cui': 'unk', 'mention': 'any anti retroviral therapy'},
 '0002_N012': {'cui': 'unk', 'mention': 'pancytopenia'},
 '0002_N013': {'cui': 'unk', 'mention': 'vomiting'},
 '0002_N014': {'cui': 'unk', 'mention': 'ddi'},
 '0002_N015': {'cui': 'unk', 'mention': 'nausea'},
 '0002_N016': {'cui': 'unk', 'mention': 'vomiting'},
 '0002_N017': {'cui': 'unk', 'mention': 'left

In [23]:
######
# Nearest neighbours calculation:
######
dd_predictions = dict()
for id in test_dict.keys():
    dd_predictions[id] = dict()
    dd_predictions[id]["pred_cui"] = [] # {'id': {'pred_cui': [] }}

# dd_predictions can be {'id': {'first candidate': [], 'top 5 candidates': [], ...}} later

CUIVectorMatrix = np.zeros((len(norm_list), embbed_size)) # len(norm_list) x embbed_size
i = 0
for cui in cui_encode.keys():
    CUIVectorMatrix[i] = cui_encode[cui]
    i += 1

In [28]:
print('\tDistance matrix calculation...')
scoreMatrix = cdist(X_pred, CUIVectorMatrix, 'cosine')  # cdist() is an optimized algo to distance calculation.
# (doc: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)

print("\tDone.")

	Distance matrix calculation...
	Done.


In [34]:
scoreMatrix[1]

array([0.09594776, 0.10041059, 0.10429328, ..., 0.09627711, 0.07196675,
       0.10031285])

In [95]:
# For each mention, find back the nearest cui vector, then attribute the associated cui:
i=0
for i, id in enumerate(test_dict.keys()):
    minScore = min(scoreMatrix[i])
    j = -1
    stopSearch = False
    for cui in cui_encode.keys():
        if stopSearch == True:
            break
        j += 1
        if scoreMatrix[i][j] == minScore:
            dd_predictions[id]["pred_cui"] = [cui]
            stopSearch = True
            break
del cui_encode

In [35]:
i=0
for i, id in enumerate(test_dict.keys()):
    min_idx = np.argmin(scoreMatrix[i])
    print(min_idx)
# outputs are same. something wrong in the training step.

207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
1633
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
1633
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
1633
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
1573
207
